In [33]:
import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role
import time
import pandas as pd
import matplotlib.pyplot as plt
from helper import *
from datetime import timedelta

In [39]:
# session, role, bucket
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

s3_bucket = sagemaker_session.default_bucket()
s3_prefix = 'MLEND-Capstone-Project'    

region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data_ind_cat".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output_ind_cat".format(s3_bucket, s3_prefix)

image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [3]:
stock_ind_cat_data = pd.read_csv('stock_indicator_data.csv',parse_dates=True, index_col=[0,1])
get_target_distribution(stock_ind_cat_data)

-1:  3268098
 0:  3220281
 1:  4107528


In [4]:
tickers = get_sp500_tickers()

freq = 'D'

# we predict for 1 day
prediction_length = 1

# we use 50 days as context length, this is the number of state updates accomplished before making predictions
context_length = 50

end_training = pd.Timestamp('2018-12-31', freq=freq)

timeseries = []
    
for ID,ticker in list(enumerate(tickers)):
    ticker = stock_ind_cat_data.loc[(slice(None), ticker), :]
    if ticker.index[0][0]<end_training:
        timeseries.append(ticker)

In [5]:
tickers = []
for ts in timeseries:
    tickers.append(ts.index[1][1])
cat = {}
for ticker in enumerate(tickers):
    cat[ticker[1]] = ticker[0]


In [29]:
end_training.freq

<Day>

In [6]:
dynamic_feat = ['Adj Close','Volume','PC1','PC2','PC3','PC4','PC5','PC6']
training_data = [
    {
        "start": str(ts.index[0][0]),
        "target": ts['target'][ts.index[0][0]:end_training].tolist(), # We use -1, because pandas indexing includes the upper bound 
        "cat" : cat[ts.index[1][1]],
        "dynamic_feat": ts[dynamic_feat][ts.index[0][0]:end_training].values.T.tolist()
        
    }
    for ts in timeseries
]
print(len(training_data))

491


In [37]:
num_test_windows = 10

test_data = [
    {
        "start": str(ts.index[0][0]),
        "target": ts['target'][ts.index[0][0]:end_training + timedelta(days=(2*k * prediction_length))].tolist(),
        "cat" : cat[ts.index[1][1]], # input stock ticker id
        "dynamic_feat": ts[dynamic_feat][ts.index[0][0]:end_training + timedelta(days=(2*k * prediction_length))].values.T.tolist()
    }
    for k in range(1, num_test_windows + 1) 
    for ts in timeseries
]
print(len(test_data))



4910


In [8]:
%%time
write_json_dataset("train_ind_cat.json", training_data)
write_json_dataset("test_ind_cat.json", test_data)

CPU times: user 1min 10s, sys: 3.63 s, total: 1min 13s
Wall time: 1min 19s


In [9]:
copy_to_s3("train_ind_cat.json", s3_data_path + "/train/train.json", s3_bucket)
copy_to_s3("test_ind_cat.json", s3_data_path + "/test/test.json", s3_bucket)

File s3://sagemaker-us-east-2-017500148529/s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data_ind_cat/train/train.json already exists.
Set override to upload anyway.

File s3://sagemaker-us-east-2-017500148529/s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data_ind_cat/test/test.json already exists.
Set override to upload anyway.



In [10]:
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")

{"start": "2010-03-16 00:00:00", "target": [1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, -1, 1, 1, 1, 1, 1...


In [7]:
estimator_ind_cat = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='MLEND-Capstone-Project',
    output_path=s3_output_path
)


In [8]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "100",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_dynamic_feat": 'auto',
}
estimator_ind_cat.set_hyperparameters(**hyperparameters)



In [9]:
%%time
data_channels = {
    "train": s3_data_path + "/train/train.json",
    "test": s3_data_path + "/test/test.json"
}

estimator_ind_cat.fit(inputs=data_channels, wait=True)


2020-06-05 04:42:59 Starting - Starting the training job...
2020-06-05 04:43:00 Starting - Launching requested ML instances......
2020-06-05 04:44:05 Starting - Preparing the instances for training...
2020-06-05 04:44:49 Downloading - Downloading input data......
2020-06-05 04:45:45 Training - Training image download completed. Training in progress.Arguments: train
[06/05/2020 04:45:48 INFO 140024126494528] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience': 

In [ ]:
predictor_ind_cat = estimator_ind_cat.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    content_type="application/json")

-------------!

In [ ]:
#predictor_indicator = sagemaker.predictor.RealTimePredictor(endpoint='MLEND-Capstone-Project-2020-06-03-23-30-23-410')

In [23]:
get_stock_prediction('AAPL', '2019-01-02',stock_ind_cat_data,predictor_ind_cat, dynamic_feat, cat['AAPL'])

/home/ec2-user/SageMaker/MLEND-Capstone-Project/helper.py:89: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  date_start = date_pred-50
/home/ec2-user/SageMaker/MLEND-Capstone-Project/helper.py:94: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  "target": pred_df['target'][date_start:date_pred-1].tolist(),
/home/ec2-user/SageMaker/MLEND-Capstone-Project/helper.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result_df['prediction'] = pred


,,Adj Close,Volume,PC1,PC2,PC3,PC4,PC5,PC6,target,prediction
Date,Ticker,,,,,,,,,,
2019-01-02,AAPL,0.03987,0.043087,-1.389812,7.484644,3.785415,-4.228811,0.443095,1.235278,-1,-1


In [25]:
date_index = pd.read_csv('test_date_index.csv')
date_index = date_index.values.reshape(252).tolist()
get_prediction_accuracy('AAPL', date_index, stock_ind_cat_data,predictor_ind_cat, dynamic_feat, cat['AAPL'])

/home/ec2-user/SageMaker/MLEND-Capstone-Project/helper.py:89: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  date_start = date_pred-50
/home/ec2-user/SageMaker/MLEND-Capstone-Project/helper.py:94: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  "target": pred_df['target'][date_start:date_pred-1].tolist(),
/home/ec2-user/SageMaker/MLEND-Capstone-Project/helper.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  result_df['prediction'] = pred


(0.746031746031746,             target  prediction
 2019-01-02      -1          -1
 2019-01-03       1           0
 2019-01-04       1           1
 2019-01-07       1           1
 2019-01-08       1           1
 2019-01-09       0           1
 2019-01-10      -1           0
 2019-01-11       1          -1
 2019-01-14       1           1
 2019-01-15       1           1
 2019-01-16       0           1
 2019-01-17       0           0
 2019-01-18      -1           0
 2019-01-22       1          -1
 2019-01-23       1           1
 2019-01-24       1           1
 2019-01-25       1           1
 2019-01-28       1           1
 2019-01-29       1           1
 2019-01-30       1           1
 2019-01-31       1           1
 2019-02-01       1           1
 2019-02-04       0           1
 2019-02-05       0           0
 2019-02-06       0           0
 2019-02-07       0           0
 2019-02-08       0           0
 2019-02-11       0           0
 2019-02-12       0           0
 2019-02-13       1  